In [59]:
#The lower the score, the more relevant the keyword is.

In [1]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS #import commen list of stopword
import en_core_web_sm
import string
from tqdm import tqdm
tqdm.pandas()
import time
import yake

In [2]:
%%time 
chunks = pd.read_json('data/arxiv-metadata-oai-snapshot.json', lines=True, chunksize = 100000)
df = pd.DataFrame()
for idx, chunk in enumerate(chunks):
    print(idx + 1)
    df = pd.concat([df,chunk[['id', 'categories','abstract']]])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
CPU times: user 1min 46s, sys: 3.45 s, total: 1min 49s
Wall time: 1min 49s


In [4]:
df = df[~df['categories'].str.contains(r'\s')]
df.abstract = df.abstract.astype(str)

In [17]:

agg_df = df.groupby('categories')['abstract'].agg(lambda x: ' '.join(x)).reset_index()

In [20]:
kw_extractor = yake.KeywordExtractor(top=10)
keywords = kw_extractor.extract_keywords(df.abstract[0])


In [18]:
agg_df = df.groupby('categories')['abstract'].agg(lambda x: ' '.join(x)).reset_index()

In [ ]:
agg_df['keywords'] = agg_df.abstract.progess_apply(lambda x: kw_extractor.extract_keywords(x.lower()))

In [24]:
agg_df.drop(columns = 'abstract').to_csv('data/masterYakeKeywords.csv', index = False)

,categories,abstract
0,astro-ph,We discuss the results from the combined IRA...
1,astro-ph.CO,We consider Brans-Dicke type nonminimally co...
2,astro-ph.EP,"We report on the discovery of HAT-P-11b, the..."
3,astro-ph.GA,The magnetic fields of our Milky Way galaxy ...
4,astro-ph.HE,We have analyzed 866 RXTE observations of th...
...,...,...
144,stat.AP,The paper deals with the study of labor mark...
145,stat.CO,The problem of the definition and the estima...
146,stat.ME,This paper considers the issue of modeling f...
147,stat.ML,We consider the problem of binary classifica...
